In [15]:
import pandas
import vincent
vincent.core.initialize_notebook()

from federal_taxes import taxes


# This makes map creation much easier and we'll store the geo_data in the function
def create_map(data, data_bind, geo_data=None, scale=None, projection=None, brew=None):
    return vincent.Map(data=data,
                       data_bind=data_bind, # Column from pandas DataFrame to plot
                       geo_data=geo_data or create_map.geo_data, 
                       scale=scale or 1000, 
                       projection=projection or 'albersUsa',
                       data_key='State', # Key holding state name from pandas DataFrame
                       map_key={'states':'properties.NAME'}, # References JSON, leave alone
                       brew=brew or 'YlGnBu')
# Bring in the geographical data for a US state map
create_map.geo_data = [{'name': 'states',
                       'url': 'map_data/us_states.topo.json',
                       'feature': 'us_states.geo'}]

## Read in the data and compute derived columns

In [2]:
# First choose the dataset, CSV file results from JSON files merged via purchasing.py
data_filename = 'mortgage.csv' 
#data_filename = 'rent.csv'

In [3]:
# Create a DataFrame from the CSV file
data = pandas.read_csv(data_filename)

In [16]:
# We'll need to create our derived field:
#  Net income subtracts off the taxes, currently just federal adjusted for household size
#  Per person then subtracts housing cost, paid with after-tax dollars, and adjusts for value and household size
data['Net Income'] = data.apply(lambda row: row['Income'] - taxes(row['Income'], row['Household Size']), axis=1)
data['Per Person'] = (data['Net Income'] - data['Housing']) * data['Value'] / data['Household Size']

In [17]:
# Check to see the data
data.sort_values(by='Net Income', ascending=False)

,State,Housing,Household Size,Income,Value,Net Income,Per Person
30,New Hampshire,9583,2.47,69888,0.9416,58376.3930,18600.752570
20,Maryland,10961,2.65,69518,0.8985,58199.5480,16016.541652
6,Connecticut,10221,2.55,67807,0.9141,56815.3395,16702.700289
45,Virginia,9911,2.60,65635,0.9690,55165.6725,16866.068328
31,New Jersey,12162,2.71,65321,0.8764,54989.6185,13850.230573
19,Massachusetts,14423,2.51,64555,0.9328,54275.6175,14810.566376
0,Alaska,11232,2.75,61749,0.9337,52250.9765,13927.061221
5,Colorado,12051,2.53,61634,0.9843,52028.3740,15553.252659
11,Hawaii,23576,2.96,60814,0.8532,51655.3290,8093.676859
7,District of Columbia,21053,2.20,61229,0.8460,51514.6315,11713.881931


# Create maps (if viewing on github, see data/*.png)

## Create map for purchasing power

In [14]:
vis = create_map(data,'Per Person')
vis.scales[0].type = 'threshold'
vis.scales[0].domain = [x*1000 for x in [0, 10, 12, 14, 16, 18, 20]]
vis.legend(title='Purchasing power')
vis.display()

## Create map for gross income

In [12]:
vis = create_map(data, 'Income')
vis.scales[0].type = 'threshold'
vis.scales[0].domain = [x * 1000 for x in [0, 40, 46, 52, 58, 64, 70]]
vis.legend(title='Gross Income')
vis.display()

## Create map for net income

In [13]:
vis = create_map(data, 'Net Income')
vis.scales[0].type = 'threshold'
vis.scales[0].domain = [x * 1000 for x in [0, 35, 40, 45, 50, 55, 60]]
vis.legend(title='Net Income')
vis.display()